In [1]:
# !pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1
# !pip uninstall -y pytorch-triton

# !pip install -U --pre triton --index-url https://download.pytorch.org/whl/nightly/cu121

Looking in indexes: https://download.pytorch.org/whl/nightly/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 18.7 MB/s eta 0:00:00:00:0100:01


In [1]:
!pip install --upgrade \
  torch torchvision torchaudio \
  --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 67.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 91.7 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 10.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 29.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 13.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 7.9 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.3 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━

In [3]:
# import os
# os.environ["TRITON_INTERPRET"] = "1"
# print(os.environ["TRITON_INTERPRET"], "\n")

import torch

print(torch.__version__)
import triton
import triton.language as tl

print(triton.__version__)

2.6.0+cu118
3.2.0


In [4]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is

stable one

In [36]:
@triton.autotune(
    configs=[
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 256,
                "BLOCK_SIZE_D2": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=1,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_K": 256,
                "BLOCK_SIZE_D2": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 128,
                "BLOCK_SIZE_D2": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 64,
                "BLOCK_SIZE_D2": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_K": 128,
                "BLOCK_SIZE_D2": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 32,
                "BLOCK_SIZE_D2": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_K": 32,
                "BLOCK_SIZE_D2": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=5,
            num_warps=2,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 32,
                "BLOCK_SIZE_K": 64,
                "BLOCK_SIZE_D2": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=5,
            num_warps=2,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 256,
                "BLOCK_SIZE_D2": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=3,
            num_warps=8,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 256,
                "BLOCK_SIZE_K": 128,
                "BLOCK_SIZE_D2": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=3,
            num_warps=8,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 256,
                "BLOCK_SIZE_K": 64,
                "BLOCK_SIZE_D2": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_K": 256,
                "BLOCK_SIZE_D2": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 128,
                "BLOCK_SIZE_D2": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 64,
                "BLOCK_SIZE_D2": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_K": 128,
                "BLOCK_SIZE_D2": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_K": 32,
                "BLOCK_SIZE_D2": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
    ],
    key=["BSIZE", "K", "d2", "L"],
)
@triton.jit
def first_pass_kernel(
    hin_ptr,
    S1s_ptr,
    U2s_ptr,
    out1_ptr,
    out2_ptr,
    BSIZE,
    K,
    d2,
    L,
    stride_hin_bsize,
    stride_hin_d2,
    stride_su_l,
    stride_su_d2,
    stride_su_k,
    stride_out_l,
    stride_out_bsize,
    stride_out_k,
    BLOCK_SIZE_BSIZE: tl.constexpr,
    BLOCK_SIZE_K: tl.constexpr,
    BLOCK_SIZE_D2: tl.constexpr,
    GROUP_SIZE_BSIZE: tl.constexpr,
):
    pid = tl.program_id(axis=1)
    batch_id = tl.program_id(axis=0)

    num_pid_bsize = tl.cdiv(BSIZE, BLOCK_SIZE_BSIZE)
    num_pid_k = tl.cdiv(K, BLOCK_SIZE_K)
    num_pid_in_group = GROUP_SIZE_BSIZE * num_pid_k
    group_id = pid // num_pid_in_group
    first_pid_bsize = group_id * GROUP_SIZE_BSIZE
    group_size_bsize = min(num_pid_bsize - first_pid_bsize, GROUP_SIZE_BSIZE)
    pid_bsize = first_pid_bsize + ((pid % num_pid_in_group) % group_size_bsize)
    pid_k = (pid % num_pid_in_group) // group_size_bsize

    offs_bsize = pid_bsize * BLOCK_SIZE_BSIZE + tl.arange(0, BLOCK_SIZE_BSIZE)
    offs_k = pid_k * BLOCK_SIZE_K + tl.arange(0, BLOCK_SIZE_K)
    offs_d2 = tl.arange(0, BLOCK_SIZE_D2)

    offs_bsize = tl.max_contiguous(
        tl.multiple_of(offs_bsize, BLOCK_SIZE_BSIZE), BLOCK_SIZE_BSIZE
    )
    offs_k = tl.max_contiguous(tl.multiple_of(offs_k, BLOCK_SIZE_K), BLOCK_SIZE_K)
    offs_d2 = tl.max_contiguous(tl.multiple_of(offs_d2, BLOCK_SIZE_D2), BLOCK_SIZE_D2)

    hin_ptrs = hin_ptr + (
        offs_bsize[:, None] * stride_hin_bsize + offs_d2[None, :] * stride_hin_d2
    )

    su_tmp = batch_id * stride_su_l + (
        offs_d2[:, None] * stride_su_d2 + offs_k[None, :] * stride_su_k
    )
    S1s_ptrs = S1s_ptr + su_tmp
    U2s_ptrs = U2s_ptr + su_tmp

    accumulator1 = tl.full(
        shape=(BLOCK_SIZE_BSIZE, BLOCK_SIZE_K), value=0.0, dtype=tl.float32
    )
    accumulator2 = tl.full(
        shape=(BLOCK_SIZE_BSIZE, BLOCK_SIZE_K), value=0.0, dtype=tl.float32
    )

    for d2_i in range(0, tl.cdiv(d2, BLOCK_SIZE_D2)):
        hin_mask = (offs_bsize[:, None] < BSIZE) & (
            offs_d2[None, :] < d2 - d2_i * BLOCK_SIZE_D2
        )
        hin = tl.load(hin_ptrs, mask=hin_mask, other=0.0)

        su_mask = (offs_d2[:, None] < d2 - d2_i * BLOCK_SIZE_D2) & (offs_k[None, :] < K)
        S1s = tl.load(S1s_ptrs, mask=su_mask, other=0.0)
        U2s = tl.load(U2s_ptrs, mask=su_mask, other=0.0)

        accumulator1 += tl.dot(hin, S1s)
        accumulator2 += tl.dot(hin, U2s)

        hin_ptrs += BLOCK_SIZE_D2 * stride_hin_d2
        S1s_ptrs += BLOCK_SIZE_D2 * stride_su_d2
        U2s_ptrs += BLOCK_SIZE_D2 * stride_su_d2

    accumulator1 = accumulator1.to(tl.float16)
    accumulator2 = accumulator2.to(tl.float16)

    out_tmp = (
        batch_id * stride_out_l
        + stride_out_bsize * offs_bsize[:, None]
        + stride_out_k * offs_k[None, :]
    )
    out1_ptrs = out1_ptr + out_tmp
    out2_ptrs = out2_ptr + out_tmp

    out_mask = (offs_bsize[:, None] < BSIZE) & (offs_k[None, :] < K)

    tl.store(out1_ptrs, accumulator1, mask=out_mask)
    tl.store(out2_ptrs, accumulator2, mask=out_mask)


def first_pass(hin, S1s, U2s):
    device = "cuda"
    assert hin.shape[1] == S1s.shape[1], "Incompatible dimensions"
    assert hin.shape[1] == U2s.shape[1], "Incompatible dimensions"
    assert hin.is_contiguous(), "Matrix A must be contiguous"
    assert S1s.is_contiguous(), "Matrix A must be contiguous"
    assert U2s.is_contiguous(), "Matrix A must be contiguous"
    assert S1s.stride() == U2s.stride(), "Matrix A must be contiguous"

    BSIZE, d2 = hin.shape
    L, _, K = S1s.shape

    out1 = torch.empty((L, BSIZE, K), dtype=torch.float16, device=device)
    out2 = torch.empty((L, BSIZE, K), dtype=torch.float16, device=device)

    stride_hin_bsize, stride_hin_d2 = hin.stride()
    stride_su_l, stride_su_d2, stride_su_k = S1s.stride()
    stride_out_l, stride_out_bsize, stride_out_k = out1.stride()

    assert out1.stride() == out2.stride(), "Matrix A must be contiguous"

    grid = lambda META: (
        L,
        triton.cdiv(BSIZE, META["BLOCK_SIZE_BSIZE"])
        * triton.cdiv(K, META["BLOCK_SIZE_K"]),
    )

    first_pass_kernel[grid](
        hin,
        S1s,
        U2s,
        out1,
        out2,
        BSIZE,
        K,
        d2,
        L,
        stride_hin_bsize,
        stride_hin_d2,
        stride_su_l,
        stride_su_d2,
        stride_su_k,
        stride_out_l,
        stride_out_bsize,
        stride_out_k,
    )

    return out1, out2


device = "cuda"

L = 4
BSIZE, d2 = 128, 16
K = 256

torch.manual_seed(0)
hin = torch.randn((BSIZE, d2), dtype=torch.float16, device=device)
S1s = torch.randn((L, d2, K), dtype=torch.float16, device=device)
U2s = torch.randn((L, d2, K), dtype=torch.float16, device=device)

start_time = time.perf_counter()
out1, out2 = first_pass(hin, S1s, U2s)
end_time = time.perf_counter()

print(f"Time taken (Python time): {end_time - start_time:.6f} seconds")

torch_output1 = (hin.unsqueeze(0).expand(L, BSIZE, d2)).bmm(S1s)
torch_output2 = (hin.unsqueeze(0).expand(L, BSIZE, d2)).bmm(U2s)

rtol = 1e-2
if torch.allclose(out1, torch_output1, atol=1e-2, rtol=rtol):
    print("✅ Triton and Torch match 1")
else:
    print("❌ Triton and Torch differ 1")

if torch.allclose(out2, torch_output2, atol=1e-2, rtol=rtol):
    print("✅ Triton and Torch match 2")
else:
    print("❌ Triton and Torch differ 2")

Time taken (Python time): 1.115365 seconds
✅ Triton and Torch match 1
✅ Triton and Torch match 2


In [23]:
@triton.autotune(
    configs=[
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 64,
                "BLOCK_SIZE_K": 256,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=1,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_D1": 32,
                "BLOCK_SIZE_K": 256,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 32,
                "BLOCK_SIZE_K": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 32,
                "BLOCK_SIZE_K": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_D1": 32,
                "BLOCK_SIZE_K": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 32,
                "BLOCK_SIZE_K": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_D1": 32,
                "BLOCK_SIZE_K": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=5,
            num_warps=2,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 32,
                "BLOCK_SIZE_D1": 32,
                "BLOCK_SIZE_K": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=5,
            num_warps=2,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 128,
                "BLOCK_SIZE_K": 256,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=3,
            num_warps=8,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 256,
                "BLOCK_SIZE_D1": 128,
                "BLOCK_SIZE_K": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=3,
            num_warps=8,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 256,
                "BLOCK_SIZE_D1": 128,
                "BLOCK_SIZE_K": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_D1": 128,
                "BLOCK_SIZE_K": 256,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 128,
                "BLOCK_SIZE_K": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 64,
                "BLOCK_SIZE_K": 64,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 64,
                "BLOCK_SIZE_D1": 64,
                "BLOCK_SIZE_K": 128,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
        triton.Config(
            {
                "BLOCK_SIZE_BSIZE": 128,
                "BLOCK_SIZE_D1": 64,
                "BLOCK_SIZE_K": 32,
                "GROUP_SIZE_BSIZE": 8,
            },
            num_stages=4,
            num_warps=4,
        ),
    ],
    key=["BSIZE", "K", "d2", "L"],
)
@triton.jit
def second_pass_kernel(
    in1_ptr,
    in2_ptr,
    U1s_ptr,
    S2s_ptr,
    bias_ptr,
    out_ptr,
    BSIZE,
    d1,
    K,
    L,
    stride_in12_l,
    stride_in12_bsize,
    stride_in12_k,
    stride_us_l,
    stride_us_k,
    stride_us_d1,
    stride_bias_bsize,
    stride_bias_d1,
    stride_out_bsize,
    stride_out_d1,
    BLOCK_SIZE_BSIZE: tl.constexpr,
    BLOCK_SIZE_D1: tl.constexpr,
    BLOCK_SIZE_K: tl.constexpr,
    GROUP_SIZE_BSIZE: tl.constexpr,
):
    pid = tl.program_id(axis=0)

    num_pid_bsize = tl.cdiv(BSIZE, BLOCK_SIZE_BSIZE)
    num_pid_d1 = tl.cdiv(d1, BLOCK_SIZE_D1)
    num_pid_in_group = GROUP_SIZE_BSIZE * num_pid_d1
    group_id = pid // num_pid_in_group
    first_pid_bsize = group_id * GROUP_SIZE_BSIZE
    GROUP_SIZE_BSIZE = min(num_pid_bsize - first_pid_bsize, GROUP_SIZE_BSIZE)
    pid_bsize = first_pid_bsize + ((pid % num_pid_in_group) % GROUP_SIZE_BSIZE)
    pid_d1 = (pid % num_pid_in_group) // GROUP_SIZE_BSIZE

    offs_bsize = pid_bsize * BLOCK_SIZE_BSIZE + tl.arange(0, BLOCK_SIZE_BSIZE)
    offs_d1 = pid_d1 * BLOCK_SIZE_D1 + tl.arange(0, BLOCK_SIZE_D1)
    offs_k = tl.arange(0, BLOCK_SIZE_K)

    offs_bsize = tl.max_contiguous(
        tl.multiple_of(offs_bsize, BLOCK_SIZE_BSIZE), BLOCK_SIZE_BSIZE
    )
    offs_d1 = tl.max_contiguous(tl.multiple_of(offs_d1, BLOCK_SIZE_D1), BLOCK_SIZE_D1)
    offs_k = tl.max_contiguous(tl.multiple_of(offs_k, BLOCK_SIZE_K), BLOCK_SIZE_K)

    in_tmp = offs_bsize[:, None] * stride_in12_bsize + offs_k[None, :] * stride_in12_k
    us_tmp = offs_k[:, None] * stride_us_k + offs_d1[None, :] * stride_us_d1

    accumulator = tl.full(
        shape=(BLOCK_SIZE_BSIZE, BLOCK_SIZE_D1), value=0.0, dtype=tl.float32
    )

    for l in range(0, L):
        l_tmp_stride = l * stride_in12_l

        in1_ptrs = l_tmp_stride + in1_ptr + in_tmp
        in2_ptrs = l_tmp_stride + in2_ptr + in_tmp

        U1s_ptrs = l_tmp_stride + U1s_ptr + us_tmp
        S2s_ptrs = l_tmp_stride + S2s_ptr + us_tmp

        for k in range(0, tl.cdiv(K, BLOCK_SIZE_K)):
            in_mask = offs_k[None, :] < K - k * BLOCK_SIZE_K
            in1 = tl.load(in1_ptrs, mask=in_mask, other=0.0)
            in2 = tl.load(in2_ptrs, mask=in_mask, other=0.0)

            us_mask = offs_k[:, None] < K - k * BLOCK_SIZE_K
            U1s = tl.load(U1s_ptrs, mask=us_mask, other=0.0)
            S2s = tl.load(S2s_ptrs, mask=us_mask, other=0.0)

            accumulator += tl.dot(in1, U1s)
            accumulator += tl.dot(in2, S2s)

            in_inc = BLOCK_SIZE_K * stride_in12_k
            in1_ptrs += in_inc
            in2_ptrs += in_inc

            us_inc = BLOCK_SIZE_K * stride_us_k
            U1s_ptrs += us_inc
            S2s_ptrs += us_inc

    accumulator = accumulator.to(tl.float16)

    bias_ptrs = bias_ptr + offs_d1[None, :] * stride_bias_d1
    bias_mask = offs_d1[None, :] < d1
    bias = tl.load(bias_ptrs, mask=bias_mask, other=0.0)

    accumulator *= 1.0 / (2.0 * L)
    accumulator += bias

    out_ptrs = (
        out_ptr
        + stride_out_bsize * offs_bsize[:, None]
        + stride_out_d1 * offs_d1[None, :]
    )
    out_mask = (offs_bsize[:, None] < BSIZE) & (offs_d1[None, :] < d1)

    tl.store(out_ptrs, accumulator, mask=out_mask)


def second_pass(in1, in2, U1s, S2s, bias):
    assert in1.shape[2] == U1s.shape[1], "Incompatible dimensions"
    assert in2.shape[2] == S2s.shape[1], "Incompatible dimensions"
    assert in1.is_contiguous(), "Matrix A must be contiguous"
    assert in2.is_contiguous(), "Matrix A must be contiguous"
    assert U1s.is_contiguous(), "Matrix A must be contiguous"
    assert S2s.is_contiguous(), "Matrix A must be contiguous"
    assert bias.is_contiguous(), "Matrix A must be contiguous"
    assert U1s.stride() == S2s.stride(), "Matrix A must be contiguous"
    assert in1.stride() == in2.stride(), "Matrix A must be contiguous"

    L, BSIZE, K = in1.shape
    _, _, d1 = U1s.shape

    out = torch.empty((BSIZE, d1), dtype=torch.float16, device=device)

    stride_in12_l, stride_in12_bsize, stride_in12_k = in1.stride()
    stride_us_l, stride_us_k, stride_us_d1 = U1s.stride()
    stride_bias_bsize, stride_bias_d1 = bias.stride()
    stride_out_bsize, stride_out_d1 = out.stride()

    grid = lambda META: (
        triton.cdiv(BSIZE, META["BLOCK_SIZE_BSIZE"])
        * triton.cdiv(d1, META["BLOCK_SIZE_D1"]),
    )

    second_pass_kernel[grid](
        in1,
        in2,
        U1s,
        S2s,
        bias,
        out,
        BSIZE,
        d1,
        K,
        L,
        stride_in12_l,
        stride_in12_bsize,
        stride_in12_k,
        stride_us_l,
        stride_us_k,
        stride_us_d1,
        stride_bias_bsize,
        stride_bias_d1,
        stride_out_bsize,
        stride_out_d1,
    )

    return out


########################################################
device = "cuda"
scale = 1
in1 = out1 * scale  # in1 -> l * bsize * k
in2 = out2 * scale  # in2 -> l * bsize * k

d1 = 128
bias = torch.randn((1, d1), dtype=torch.float16, device=device) * scale

U1s = torch.randn((L, K, d1), dtype=torch.float16, device=device) * scale
S2s = torch.randn((L, K, d1), dtype=torch.float16, device=device) * scale

triton_output = second_pass(in1, in2, U1s, S2s, bias)
torch_output = (in1.bmm(U1s).mean(0) / 2) + (in2.bmm(S2s).mean(0) / 2) + bias

rtol = 10e-1
if torch.allclose(triton_output, torch_output, atol=1e-2, rtol=rtol):
    print("✅ Triton and Torch match")
else:
    print("❌ Triton and Torch differ")

✅ Triton and Torch match


In [25]:
def forward(hin, S1s, U2s, bias, U1s, S2s):
    in1, in2 = first_pass(hin, S1s, U2s)
    triton_output = second_pass(in1, in2, U1s, S2s, bias)
    return triton_output

In [26]:
torch.manual_seed(0)

L = 4
BSIZE, d2 = 128, 16
K = 256
d1 = 128

hin = torch.randn((BSIZE, d2), dtype=torch.float16, device=device)
S1s = torch.randn((L, d2, K), dtype=torch.float16, device=device)
S2s = torch.randn((L, K, d1), dtype=torch.float16, device=device)
U1s = torch.randn((L, K, d1), dtype=torch.float16, device=device)
U2s = torch.randn((L, d2, K), dtype=torch.float16, device=device)
bias = torch.randn((1, d1), dtype=torch.float16, device=device)

triton_out = forward(hin, S1s, U2s, bias, U1s, S2s)

inputt = hin.unsqueeze(0).expand(L, hin.shape[0], hin.shape[1])
torch_output = (
    ((inputt.bmm(S1s)).bmm(U1s)).mean(0) / 2
    + ((inputt.bmm(U2s)).bmm(S2s)).mean(0) / 2
    + bias
)

rtol = 10e-1
if torch.allclose(triton_out, torch_output, atol=1e-2, rtol=rtol):
    print("✅ Triton and Torch match 1")
else:
    print("❌ Triton and Torch differ 1")

✅ Triton and Torch match 1


In [29]:
import torch
import torch.utils.benchmark as benchmark
import time


def forward_torch(hin, S1s, U2s, bias, U1s, S2s):
    L = S1s.shape[0]
    inputt = hin.unsqueeze(0).expand(L, hin.shape[0], hin.shape[1])
    return (
        ((inputt.bmm(S1s)).bmm(U1s)).mean(0) / 2
        + ((inputt.bmm(U2s)).bmm(S2s)).mean(0) / 2
        + bias
    )


# Compile the PyTorch function
forward_torch_compiled = torch.compile(forward_torch)

# Compile the original forward function with full graph mode
forward_compiled = torch.compile(forward, fullgraph=True)


def benchmark_functions(hin, S1s, U2s, bias, U1s, S2s, num_warmup=10, num_runs=100):
    # Move all tensors to CUDA
    hin = hin.cuda()
    S1s = S1s.cuda()
    S2s = S2s.cuda()
    U1s = U1s.cuda()
    U2s = U2s.cuda()
    bias = bias.cuda()

    # Verify results match
    triton_out = forward(hin, S1s, U2s, bias, U1s, S2s)
    torch_out = forward_torch(hin, S1s, U2s, bias, U1s, S2s)
    compiled_torch_out = forward_torch_compiled(hin, S1s, U2s, bias, U1s, S2s)
    compiled_triton_out = forward_compiled(hin, S1s, U2s, bias, U1s, S2s)

    print(
        f"Triton and Torch match: {torch.allclose(triton_out, torch_out, atol=1e-2, rtol=1e-2)}"
    )
    print(
        f"Triton and Compiled Torch match: {torch.allclose(triton_out, compiled_torch_out, atol=1e-2, rtol=1e-2)}"
    )
    print(
        f"Triton and Compiled Triton match: {torch.allclose(triton_out, compiled_triton_out, atol=1e-2, rtol=1e-2)}"
    )

    # Warmup
    print("Warming up...")
    for _ in range(num_warmup):
        _ = forward(hin, S1s, U2s, bias, U1s, S2s)
        _ = forward_torch(hin, S1s, U2s, bias, U1s, S2s)
        _ = forward_torch_compiled(hin, S1s, U2s, bias, U1s, S2s)
        _ = forward_compiled(hin, S1s, U2s, bias, U1s, S2s)

    # Synchronize before timing
    torch.cuda.synchronize()

    # Benchmark using torch.utils.benchmark
    timer_triton = benchmark.Timer(
        stmt="forward(hin, S1s, U2s, bias, U1s, S2s)",
        globals={
            "forward": forward,
            "hin": hin,
            "S1s": S1s,
            "U2s": U2s,
            "bias": bias,
            "U1s": U1s,
            "S2s": S2s,
        },
    )

    timer_torch = benchmark.Timer(
        stmt="forward_torch(hin, S1s, U2s, bias, U1s, S2s)",
        globals={
            "forward_torch": forward_torch,
            "hin": hin,
            "S1s": S1s,
            "U2s": U2s,
            "bias": bias,
            "U1s": U1s,
            "S2s": S2s,
        },
    )

    timer_compiled_torch = benchmark.Timer(
        stmt="forward_torch_compiled(hin, S1s, U2s, bias, U1s, S2s)",
        globals={
            "forward_torch_compiled": forward_torch_compiled,
            "hin": hin,
            "S1s": S1s,
            "U2s": U2s,
            "bias": bias,
            "U1s": U1s,
            "S2s": S2s,
        },
    )

    timer_compiled_triton = benchmark.Timer(
        stmt="forward_compiled(hin, S1s, U2s, bias, U1s, S2s)",
        globals={
            "forward_compiled": forward_compiled,
            "hin": hin,
            "S1s": S1s,
            "U2s": U2s,
            "bias": bias,
            "U1s": U1s,
            "S2s": S2s,
        },
    )

    # Run benchmarks
    print("Running benchmarks...")
    result_triton = timer_triton.timeit(num_runs)
    result_torch = timer_torch.timeit(num_runs)
    result_compiled_torch = timer_compiled_torch.timeit(num_runs)
    result_compiled_triton = timer_compiled_triton.timeit(num_runs)

    # Print results
    print(f"\nTriton implementation: {result_triton}")
    print(f"PyTorch implementation: {result_torch}")
    print(f"Compiled PyTorch implementation: {result_compiled_torch}")
    print(f"Compiled Triton implementation: {result_compiled_triton}")

    # Calculate speedups
    triton_time = result_triton.mean
    torch_time = result_torch.mean
    compiled_torch_time = result_compiled_torch.mean
    compiled_triton_time = result_compiled_triton.mean

    print(f"\nSpeedup of Triton vs PyTorch: {torch_time / triton_time:.2f}x")
    print(
        f"Speedup of Triton vs Compiled PyTorch: {compiled_torch_time / triton_time:.2f}x"
    )
    print(
        f"Speedup of Compiled PyTorch vs PyTorch: {torch_time / compiled_torch_time:.2f}x"
    )
    print(
        f"Speedup of Compiled Triton vs Triton: {triton_time / compiled_triton_time:.2f}x"
    )
    print(
        f"Speedup of Compiled Triton vs Compiled PyTorch: {compiled_torch_time / compiled_triton_time:.2f}x"
    )


if __name__ == "__main__":
    # Set random seed for reproducibility
    torch.manual_seed(0)

    # Create test data
    L = 4
    BSIZE, d2 = 128, 16
    K = 256
    d1 = 128
    hin = torch.randn((BSIZE, d2), dtype=torch.float16)
    S1s = torch.randn((L, d2, K), dtype=torch.float16)
    S2s = torch.randn((L, K, d1), dtype=torch.float16)
    U1s = torch.randn((L, K, d1), dtype=torch.float16)
    U2s = torch.randn((L, d2, K), dtype=torch.float16)
    bias = torch.randn((1, d1), dtype=torch.float16)

    # Run benchmarks
    benchmark_functions(hin, S1s, U2s, bias, U1s, S2s)

Triton and Torch match: False
Triton and Compiled Torch match: False
Triton and Compiled Triton match: True
Warming up...
Running benchmarks...

Triton implementation: <torch.utils.benchmark.utils.common.Measurement object at 0x785214f3acb0>
forward(hin, S1s, U2s, bias, U1s, S2s)
  218.84 us
  1 measurement, 100 runs , 1 thread
PyTorch implementation: <torch.utils.benchmark.utils.common.Measurement object at 0x785214f3b280>
forward_torch(hin, S1s, U2s, bias, U1s, S2s)
  120.91 us
  1 measurement, 100 runs , 1 thread
Compiled PyTorch implementation: <torch.utils.benchmark.utils.common.Measurement object at 0x785232794910>
forward_torch_compiled(hin, S1s, U2s, bias, U1s, S2s)
  143.11 us
  1 measurement, 100 runs , 1 thread
Compiled Triton implementation: <torch.utils.benchmark.utils.common.Measurement object at 0x7851f4e7a290>
forward_compiled(hin, S1s, U2s, bias, U1s, S2s)
  93.41 us
  1 measurement, 100 runs , 1 thread

Speedup of Triton vs PyTorch: 0.55x
Speedup of Triton vs Compiled